# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
stocks = sp500['Symbol'].tolist()
index = stocks.index('BRK.B')
stocks[index] = 'BRK-B'
index = stocks.index('BF.B')
stocks[index] = 'BF-B'

stocks

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ADBE',
 'AMD',
 'AES',
 'AFL',
 'A',
 'APD',
 'ABNB',
 'AKAM',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'AON',
 'APA',
 'APO',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'AXON',
 'BKR',
 'BALL',
 'BAC',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'XYZ',
 'BK',
 'BA',
 'BKNG',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF-B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'COIN',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',


## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [ ]:
# # Function sourced from 
# # https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
# def chunks(lst, n):
#     """Yield successive n-sized chunks from lst."""
#     for i in range(0, len(lst), n):
#         yield lst[i:i + n]   
        
# symbol_groups = list(chunks(stocks['Ticker'], 100))
# symbol_strings = []
# for i in range(0, len(symbol_groups)):
#     symbol_strings.append(','.join(symbol_groups[i]))
# #     print(symbol_strings[i])

# my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [20]:
import yfinance as yf

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)

for stock in stocks:
        data = yf.Ticker(stock).fast_info    

        if 'lastPrice' in data and 'yearChange' in data:
            final_dataframe.loc[len(final_dataframe)] = [
                stock,
                data['lastPrice'],
                data['yearChange']* 100,
                'N/A'
            ]
            
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,153.300003,24.270430,N/A
1,AOS,70.830002,-10.193991,N/A
2,ABT,134.279999,24.564003,N/A
3,ABBV,198.050003,4.138190,N/A
4,ACN,239.199997,-23.811949,N/A
...,...,...,...,...
497,XYL,142.649994,11.882348,N/A
498,YUM,141.279999,4.165740,N/A
499,ZBRA,317.660004,-0.836609,N/A
500,ZBH,99.260002,-7.233643,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [16]:
final_dataframe.sort_values("One-Year Price Return", ascending=False, inplace=True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,359,PLTR,186.960007,5.363513,N/A
1,210,GEV,649.090027,2.678814,N/A
2,432,TPR,109.089996,1.909842,N/A
3,474,VST,202.119995,1.592612,N/A
4,53,AXON,842.500000,1.289216,N/A
5,459,UAL,89.290001,1.209055,N/A
6,261,JBL,224.339996,1.198981,N/A
7,152,DASH,259.190002,1.089568,N/A
8,71,AVGO,304.970001,1.052012,N/A
9,402,RCL,301.440002,0.959184,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [17]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the size of your portfolio:")


portfolio_input()
print(portfolio_size)


1000000


In [18]:
position_size = float(portfolio_size)/len(final_dataframe.index)
print(position_size)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Price"])

final_dataframe

20000.0


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,359,PLTR,186.960007,5.363513,106
1,210,GEV,649.090027,2.678814,30
2,432,TPR,109.089996,1.909842,183
3,474,VST,202.119995,1.592612,98
4,53,AXON,842.500000,1.289216,23
5,459,UAL,89.290001,1.209055,223
6,261,JBL,224.339996,1.198981,89
7,152,DASH,259.190002,1.089568,77
8,71,AVGO,304.970001,1.052012,65
9,402,RCL,301.440002,0.959184,66


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [21]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'Average Return Percentile'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for stock in stocks:
    data = yf.Ticker(stock).info
    
    hqm_dataframe.loc[len(hqm_dataframe)] = [
            stock,
            data['currentPrice'],
            'N/A',
            data['fiftyTwoWeekChangePercent'],
            'One-Year Retrun Percentile',
            data['twoHundredDayAverageChangePercent'],
            'Six-Month Return Percentile',
            data['fiftyDayAverageChangePercent'],
            'Three-Month Return Percentile',
            'N/A'
    ]

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Average Return Percentile
0,MMM,153.30,N/A,24.270428,One-Year Retrun Percentile,0.077693,Six-Month Return Percentile,0.021756,Three-Month Return Percentile,N/A
1,AOS,70.83,N/A,-10.193992,One-Year Retrun Percentile,0.031387,Six-Month Return Percentile,0.049215,Three-Month Return Percentile,N/A
2,ABT,134.28,N/A,24.564003,One-Year Retrun Percentile,0.064694,Six-Month Return Percentile,0.019412,Three-Month Return Percentile,N/A
3,ABBV,198.05,N/A,4.138184,One-Year Retrun Percentile,0.053309,Six-Month Return Percentile,0.044422,Three-Month Return Percentile,N/A
4,ACN,239.20,N/A,-23.811949,One-Year Retrun Percentile,-0.273010,Six-Month Return Percentile,-0.179985,Three-Month Return Percentile,N/A
...,...,...,...,...,...,...,...,...,...,...
497,XYL,142.65,N/A,11.882353,One-Year Retrun Percentile,0.145368,Six-Month Return Percentile,0.088004,Three-Month Return Percentile,N/A
498,YUM,141.28,N/A,4.165745,One-Year Retrun Percentile,-0.010965,Six-Month Return Percentile,-0.028708,Three-Month Return Percentile,N/A
499,ZBRA,317.66,N/A,-0.836611,One-Year Retrun Percentile,-0.038270,Six-Month Return Percentile,0.016756,Three-Month Return Percentile,N/A
500,ZBH,99.26,N/A,-7.233644,One-Year Retrun Percentile,-0.029771,Six-Month Return Percentile,0.063146,Three-Month Return Percentile,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [22]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Average Return Percentile
0,MMM,153.30,N/A,24.270428,0.721116,0.077693,0.681275,0.021756,0.577689,N/A
1,AOS,70.83,N/A,-10.193992,0.247012,0.031387,0.531873,0.049215,0.701195,N/A
2,ABT,134.28,N/A,24.564003,0.7251,0.064694,0.635458,0.019412,0.557769,N/A
3,ABBV,198.05,N/A,4.138184,0.454183,0.053309,0.601594,0.044422,0.663347,N/A
4,ACN,239.20,N/A,-23.811949,0.095618,-0.273010,0.035857,-0.179985,0.033865,N/A
...,...,...,...,...,...,...,...,...,...,...
497,XYL,142.65,N/A,11.882353,0.549801,0.145368,0.830677,0.088004,0.874502,N/A
498,YUM,141.28,N/A,4.165745,0.456175,-0.010965,0.410359,-0.028708,0.314741,N/A
499,ZBRA,317.66,N/A,-0.836611,0.37251,-0.038270,0.338645,0.016756,0.541833,N/A
500,ZBH,99.26,N/A,-7.233644,0.302789,-0.029771,0.356574,0.063146,0.780876,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [23]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'Average Return Percentile'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Average Return Percentile
0,MMM,153.30,N/A,24.270428,0.721116,0.077693,0.681275,0.021756,0.577689,0.660027
1,AOS,70.83,N/A,-10.193992,0.247012,0.031387,0.531873,0.049215,0.701195,0.49336
2,ABT,134.28,N/A,24.564003,0.7251,0.064694,0.635458,0.019412,0.557769,0.639442
3,ABBV,198.05,N/A,4.138184,0.454183,0.053309,0.601594,0.044422,0.663347,0.573041
4,ACN,239.20,N/A,-23.811949,0.095618,-0.273010,0.035857,-0.179985,0.033865,0.055113
...,...,...,...,...,...,...,...,...,...,...
497,XYL,142.65,N/A,11.882353,0.549801,0.145368,0.830677,0.088004,0.874502,0.75166
498,YUM,141.28,N/A,4.165745,0.456175,-0.010965,0.410359,-0.028708,0.314741,0.393758
499,ZBRA,317.66,N/A,-0.836611,0.37251,-0.038270,0.338645,0.016756,0.541833,0.417663
500,ZBH,99.26,N/A,-7.233644,0.302789,-0.029771,0.356574,0.063146,0.780876,0.48008


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [24]:
hqm_dataframe.sort_values('Average Return Percentile', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True)
hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Average Return Percentile
0,359,PLTR,186.960,N/A,536.351260,1.0,0.870626,1.0,0.287243,0.996016,0.998672
1,210,GEV,649.090,N/A,267.881440,0.998008,0.630996,0.998008,0.186324,0.986056,0.994024
2,432,TPR,109.090,N/A,190.984270,0.996016,0.457033,0.994024,0.163237,0.972112,0.987384
3,43,ANET,139.180,N/A,64.549410,0.948207,0.404294,0.982072,0.323220,1.0,0.97676
4,355,ORCL,250.050,N/A,88.603100,0.972112,0.408594,0.986056,0.135589,0.962151,0.97344
5,489,WDC,74.970,N/A,64.295700,0.946215,0.461996,0.996016,0.168594,0.978088,0.97344
6,124,GLW,65.770,N/A,67.311120,0.958167,0.335901,0.968127,0.215294,0.994024,0.97344
7,161,EBAY,93.140,N/A,66.887650,0.952191,0.348282,0.972112,0.175013,0.984064,0.969456
8,71,AVGO,304.970,N/A,105.201170,0.984064,0.402008,0.978088,0.121053,0.944223,0.968792
9,346,NVDA,182.700,N/A,67.583930,0.960159,0.344678,0.97012,0.147541,0.968127,0.966135


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [26]:
portfolio_input()

position_size = float(portfolio_size)/len(hqm_dataframe.index)
print(position_size)
for i in range(0, len(hqm_dataframe)):
    hqm_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/hqm_dataframe.loc[i, "Price"])

hqm_dataframe

200000.0


,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,Average Return Percentile
0,359,PLTR,186.960,1069,536.351260,1.0,0.870626,1.0,0.287243,0.996016,0.998672
1,210,GEV,649.090,308,267.881440,0.998008,0.630996,0.998008,0.186324,0.986056,0.994024
2,432,TPR,109.090,1833,190.984270,0.996016,0.457033,0.994024,0.163237,0.972112,0.987384
3,43,ANET,139.180,1436,64.549410,0.948207,0.404294,0.982072,0.323220,1.0,0.97676
4,355,ORCL,250.050,799,88.603100,0.972112,0.408594,0.986056,0.135589,0.962151,0.97344
5,489,WDC,74.970,2667,64.295700,0.946215,0.461996,0.996016,0.168594,0.978088,0.97344
6,124,GLW,65.770,3040,67.311120,0.958167,0.335901,0.968127,0.215294,0.994024,0.97344
7,161,EBAY,93.140,2147,66.887650,0.952191,0.348282,0.972112,0.175013,0.984064,0.969456
8,71,AVGO,304.970,655,105.201170,0.984064,0.402008,0.978088,0.121053,0.944223,0.968792
9,346,NVDA,182.700,1094,67.583930,0.960159,0.344678,0.97012,0.147541,0.968127,0.966135


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [28]:
writer = pd.ExcelWriter('momentum_strategy_learning.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [31]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [32]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['Average Return Percentile', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [33]:
writer.close()